In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import MiniBatchKMeans
import numpy as np
import pandas as pd
import csv
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer  
import os.path
import sys

In [ ]:
def getMatchByTfidf(corpus, query):
    tfidf = TfidfVectorizer()
    tfs = tfidf.fit_transform(corpus)

    response = tfidf.transform([query])
#     feature_names = tfidf.get_feature_names()
#     for col in response.nonzero()[1]:
#         print( feature_names[col], ' - ', response[0, col])
#     print(np.amax(np.sum(tfs.toarray()[:,response.nonzero()[1]], axis=1)))
#     print(tfs.toarray()[:,response.nonzero()[1]])
    # Print the highest tfidf score
    score = np.amax(np.sum(tfs.toarray()[:,response.nonzero()[1]], axis=1))
    print(score)
    # best : the location in corpus of the best score
    best = np.argmax(np.sum(tfs.toarray()[:,response.nonzero()[1]], axis=1))
#     if not best:
#         # if best score is not zero, then return the corpus
#         return 'Exception: No matching'
    return ( corpus[best], score )

if __name__ == '__main__':
    text1 = 'Cosmoprof North America Las Vegas(Cosmoprof)'
    text2 = 'National Safety Council Congress & Expo Anaheim 2016(NSC Congress & Expo)'
    text3 = 'NSC - National Safety Council Congress & Exposition'
    text5 = 'International Consumer Electronics Show(2013 International CES)'

    query = 'congress International CES - Consumer Electronics Show - the Source for Consumer Technologies'
    query2 = 'safety    2016'
    query3 = ''
    query4 = 'you are a ass hole congress'

    corpus = (text1, text2, text3, text5)

    print(getMatchByTfidf(corpus, query2))
    print(getMatchByTfidf(corpus, query3))
    print(getMatchByTfidf(corpus, query))
    print(getMatchByTfidf(corpus, query4))


In [ ]:
def checkFileExist(filePath):
    if os.path.isfile(filePath):
        return
    print( "File : \'{}\' doesn't exist.".format(filePath) )
    sys.exit()
checkFileExist('data/exhibitMaifn.csv')

In [ ]:
my_data = pd.read_csv('fuzzyTable.csv')
for key in set(my_data['FuzzyKey']):
    tmp = my_data.loc[ my_data['FuzzyKey'] == key ]
    corpus = tuple(tmp['EXHIBITION_NAME'])
    query = tuple(tmp['展覽名稱'])[0]
    tmpList = [key,
               query,
               getMatchByTfidf(corpus, query)[0],
               getMatchByTfidf(corpus, query)[1]]
    with open("output.csv", "a") as fp:
        wr = csv.writer(fp, dialect='excel', quotechar = '"')
        wr.writerow(tmpList)
    print('query : %s\nresult : %s' % 
        ( query, getMatchByTfidf(corpus, query)[0] ) )


In [ ]:
my_data = pd.read_csv('data/jetro1718.csv', encoding = "ISO-8859-1")
corpus = tuple(my_data['query'])
print('Vectorizing...')
tfidf = TfidfVectorizer(stop_words=None)
tfs = tfidf.fit_transform(corpus)


simMat = (tfs * tfs.T).A


print('Add into key...')
for index in range( len(corpus) ):
    print('Adding group %d' % index)
    a = np.where( simMat[index] > 0.5 )[0]
    tmpDF = pd.concat( [pd.DataFrame( ['group' + str(index)]*len(a) ),
                        pd.DataFrame( [corpus[i] for i in a] ),
                        pd.DataFrame( [simMat[index, i] for i in a] ),
                        pd.DataFrame( [my_data['dataId'].loc[i] for i in a] ),
                        pd.DataFrame( [my_data['idGroup'].loc[i] for i in a] )],
                        axis = 1 )
    with open('groupOutputSite_0118.csv', 'a') as f:
        tmpDF.to_csv(f, header=False)

In [ ]:
my_data = pd.read_csv('data/exhibitMain.csv', encoding = "ISO-8859-1")
corpus = tuple(my_data['text'])
tfidf = TfidfVectorizer(stop_words=None)
print('Vectorizing...')
tfs = tfidf.fit_transform(corpus)

queryTable = pd.read_csv('data/Jetro2017.csv')

file = 'output2017.csv'
with open(file=outputFile, "a") as fp:
    wr = csv.writer(fp, dialect='excel', quotechar = '"')
    wr.writerow(['dataId', 'query', 'mainText', 'showId', 'Score'])
for index in range(len(queryTable)):
    print('Querying %d...' % index)
    dataId = queryTable.loc[index][0]
    query = queryTable.loc[index][1]
    response = tfidf.transform([query])
    score = np.amax(np.sum(tfs.toarray()[:,response.nonzero()[1]], axis=1))
    best = np.argmax(np.sum(tfs.toarray()[:,response.nonzero()[1]], axis=1))
    tmpList = [dataId,
               query,
               my_data.loc[best][1],
               int(my_data.loc[best][0]),
               score]
    with open(file=outputFile, "a") as fp:
        wr = csv.writer(fp, dialect='excel', quotechar = '"')
        wr.writerow(tmpList)


In [ ]:
def matchEngine(tfidf, tfs, query):
    queryCleaned = cleanWords(query)
    response = tfidf.transform([queryCleaned])
    score = np.amax(np.sum(tfs.toarray()[:,response.nonzero()[1]], axis=1))
    score_std = score / len(response.nonzero()[1])
    best = np.argmax(np.sum(tfs.toarray()[:,response.nonzero()[1]], axis=1))
    matchResult = my_data.loc[best][1]
    hitw_tup1 = get_hitw(tfidf, queryCleaned, 1)
    hitw_tup2 = get_hitw(tfidf, queryCleaned, 2)
    hitw1 = hitw_tup1[1]
    hitw2 = hitw_tup2[1]
    hit1 = hitw_tup1[0]
    hit2 = hitw_tup2[0]
    contain_hit1 = containTerm(hit1, matchResult)
    contain_hit2 = containTerm(hit2, matchResult)
    
    tmpList = [query,
               matchResult,
               int(my_data.loc[best][0]),
               score,
               score_std,
               hit1,
               hitw1,
               contain_hit1,
               hit2,
               hitw2,
               contain_hit2]
    return tmpList

# print('Vectorizing...')
# tfidf = TfidfVectorizer()
# tfs = tfidf.fit_transform(corpusProcessed)

# query = 'Industrial Automation / HANNOVER MESSE - Leading Trade Fair for Process, Factory and Building Automation Systems and Solutions'
# query = 'hannover automation spring hannover messe'
# query = 'MDA-Motion, drives& Automation / HANNOVER MESSE - Leading Trade Fair for Power Transmission and Control'

# query = 'MAGICONLINE veryRareWord- PROJECT'
# query = cleanWords(query)
# matchEngine(tfidf, tfs, query)

In [ ]:
queryFile = './data/Jetro0108_1_.csv'
exhibitMainFile = 'data/exhibitMain0108.csv'
outputFile = 'output/output0108_1_.csv'

queryTable = pd.read_csv(queryFile)
my_data = pd.read_csv(exhibitMainFile, encoding = "ISO-8859-1")
corpusOriginal = tuple(my_data['text'])
print('Cleaning corpus...')
# corpusProcessed = map(cleanWords, corpusOriginal)
corpusProcessed = [cleanWords(x) for x in corpusOriginal]

print('Vectorizing...')
tfidf = TfidfVectorizer()
tfs = tfidf.fit_transform(corpusProcessed)

with open(outputFile, "a") as fp:
    wr = csv.writer(fp, dialect='excel', quotechar = '"')
    wr.writerow(['dataId',
                 'query',
                 'matchResult',
                 'showId',
                 'Score',
                 'ScoreStd',
                 'hit1',
                 'hitw1',
                 'contain_hit1',
                 'hit2',
                 'hitw2',
                 'contain_hit2'])
for index in range(len(queryTable)):
    print('Querying %d...' % index)
    dataId = queryTable.loc[index]['dataId']
    query = queryTable.loc[index]['query']
    tmpList = matchEngine(tfidf, tfs, query)
    tmpList = [dataId] + tmpList
    with open(outputFile, "a") as fp:
        wr = csv.writer(fp, dialect='excel', quotechar = '"')
        wr.writerow(tmpList)

In [ ]:
def containTerm(term, mainResult):
    mainResult = cleanWords(mainResult)
    return term in mainResult

mainResult = 'MAGICONLINE veryRareWord- PROJECT'
containTerm('magica', mainResult)

In [ ]:
def get_hitw(tfidf, query, order):
    # hitw : high informative term weight
    unigrams = query.split(' ')
    dataDict = dict()
    for i in range(len(unigrams)):
        loc0 = tfidf.vocabulary_.get(unigrams[i])
        if loc0 == None:
            continue
        dataDict[unigrams[i]] = tfidf.idf_[loc0]
    s = [(k, dataDict[k]) for k in sorted(dataDict, key=dataDict.get, reverse=True)]
    # tw_tup : term-weight tuple
    if len(s) == 0:
        return ('NoWord', 0.0)
    if order > len(s):
        order = len(s)
    tw_tup = s[order - 1]
    return tw_tup

query = 'Industrial Automation / HANNOVER MESSE - Leading Trade Fair for Process, Factory and Building Automation Systems and Solutions'
query = 'hannover automation spring hannover messe'
query = 'MDA-Motion, drives& Automation / HANNOVER MESSE - Leading Trade Fair for Power Transmission and Control'

query = 'MAGICONLINE veryRareWord- PROJECT'
query = cleanWords(query)

get_hitw(tfidf, query, 3)


In [ ]:
query = 'MDA-Motion, CES fairs Drive drives& Automation / HANNOVER MESSE - Leading Trade Fair for Power Transmission and Control'

text1 = 'Cosmoprof North America Las Vegas(Cosmoprof)'
text2 = 'National Safety Council Congress & Expo Anaheim 2016(NSC Congress & Expo)'
text3 = 'NSC - National Safety Council Congress & Exposition'
text5 = 'International Consumer Electronics Show(2013 International CES)'

text4 = 'congress International CES - Consumer Electronics Show - the Source for Consumer Technologies'

def cleanWords(query):
    try:
        # Get tokens
        counterTemp = CountVectorizer(ngram_range=(1,1))
        langModTemp = counterTemp.fit_transform([query])
        unigrams = counterTemp.get_feature_names()
        # Remove stopwords
        filtered_words = [word for word in unigrams if word not in stopwords.words('english')]
        # Lemmatize or stemming
        # This could be altered by st.stem(word)
        # or
        # snowball_stemmer = SnowballStemmer('english')  
        # snowball_stemmer.stem(word)
        wnl = WordNetLemmatizer()
        cleanWords = [wnl.lemmatize(word, "v") for word in filtered_words]
        # Concatenate
        sentence = ' '.join(cleanWords)
    except ValueError:
        print('perhaps the documents only contain stop words')
        return ' '
    return sentence

cleanWords(text4)
